In [6]:
import pandas as pd
import os
from itertools import permutations
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

In [7]:
# Gather 2022 Details 
data_dir = 'C:/Users/harry/Documents/data/Project_Node/2022-08-14-171850/'
person = pd.read_csv(os.path.join(data_dir, 'PersonHistorical.csv'))

# Filter Person ID by Internal Netherland Office people 
filtered_person_df = person[(person['Country']=='Netherlands') & (person['WorkerType']=='Employee') & \
                            (person['Domain']=='CUSHWAKE.COM') & (person['ActiveStatus']=='Yes') & \
                            (person['IsInternal']==True) ]

# Helix Office
# filtered_person_df = filtered_person_df[filtered_person_df['Office'].isin(['C - NLD - Utrecht - Franz-Lisztplantsoen 100', 
#                                                       'C - NLD - Naarden - Gooimeer 6-32'])]

# Gather 2021 Details
data_dir_2 = 'C:/Users/harry/Documents/data/Project_Node/2021/'
person_2021 = pd.read_csv(os.path.join(data_dir_2, 'PersonHistorical.csv'))

# Filter Person ID by Internal Netherland Office people 
filtered_person_2021 = person_2021[(person_2021['Country']=='Netherlands') & (person_2021['WorkerType']=='Employee') & \
                            (person_2021['Domain']=='CUSHWAKE.COM') & (person_2021['ActiveStatus']=='Yes') & \
                            (person_2021['IsInternal']==True) ]

# Helix Office
# filtered_person_2021 = filtered_person_2021[filtered_person_2021['Office'].isin(['C - NLD - Utrecht - Franz-Lisztplantsoen 100', 
#                                                       'C - NLD - Naarden - Gooimeer 6-32'])]

email_list_20212022 = filtered_person_2021.merge(filtered_person_df, on='EmailAddress', how='inner')['EmailAddress'].unique()

In [36]:
pd.set_option("display.max_rows", False, "display.max_columns", None)


In [9]:
report_dir = 'C:/Users/harry/Documents/data/Project_Node/report/'

Calls = pd.read_csv(os.path.join(report_dir, 'Calls_Processed_01022022_31032022.csv'))
Messages = pd.read_csv(os.path.join(report_dir, 'IMS_Processed_01022022_31032022.csv'))
Mails = pd.read_csv(os.path.join(report_dir, 'Mails_Processed_01022022_31032022.csv'))
Meetings = pd.read_csv(os.path.join(report_dir, 'Meetings_Processed_01022022_31032022.csv'))

total_comms_df = Calls.merge(Mails, on=['EmailAddress1', 'EmailAddress2'], how='outer'
           ).merge(Messages, on=['EmailAddress1', 'EmailAddress2'], how='outer'
                  ).merge(Meetings, on=['EmailAddress1', 'EmailAddress2'], how='outer')

total_comms_df.fillna(0, inplace=True)

for col in ['TotalCallMinutes', 'TotalSpentinMinutesMails', 'TotalSpentinMinutesIMS', 'TotalSpentinMinutesMeetings']:
    x_scaler = MinMaxScaler(feature_range=(0, 100))
    x_scaler.fit(total_comms_df[col].values.reshape(-1, 1))
    total_comms_df[col+'Scaled'] = pd.DataFrame(x_scaler.transform(total_comms_df[col].values.reshape(-1, 1)))

mail_weight = 0.25
message_weight = 0.25
calls_weight = 0.25
meetings_weight = 0.25

total_comms_df['connectivity_score'] = mail_weight * total_comms_df['TotalSpentinMinutesMailsScaled'] + \
                                        message_weight * total_comms_df['TotalSpentinMinutesIMSScaled'] + \
                                        calls_weight * total_comms_df['TotalCallMinutesScaled'] + \
                                        meetings_weight * total_comms_df['TotalSpentinMinutesMeetingsScaled']

filtered_person_df.sort_values(by='StartDate', ascending=False, inplace=True)

df = filtered_person_df[['EmailAddress', 'Office']].drop_duplicates(subset=['EmailAddress'], keep='first')
df.columns = ['EmailAddress1', 'Office1']
total_comms_df2 = total_comms_df.merge(df, on='EmailAddress1', how='left')

df.columns = ['EmailAddress2', 'Office2']
total_comms_df3 = total_comms_df2.merge(df, on='EmailAddress2', how='left')
total_comms_df3.to_csv(os.path.join(report_dir, 'OverallCommunication_Processed_01022022_31032022.csv'), index=False)

In [24]:
total_comms_df3.to_csv(os.path.join(report_dir, 'OverallCommunication_Processed_01032022_01082022.csv'), index=False)

In [ ]:
# Part 2 - Helix Study

In [30]:
report_dir = 'C:/Users/harry/Documents/data/Project_Node/report/Mar_Aug_2022_processed/'

total_comms_df = pd.read_csv(os.path.join(report_dir, 'OverallCommunication_Processed_01032022_01082022.csv'))

report_dir_2 = 'C:/Users/harry/Documents/data/Project_Node/report/Mar_Aug_2021_processed/'
total_comms_2021 = pd.read_csv(os.path.join(report_dir_2, 'OverallCommunication_Processed_01032021_01082021.csv'))

In [31]:
total_comms_heliex_2022 = total_comms_df[(total_comms_df['EmailAddress1'].isin(email_list_20212022)) & 
               total_comms_df['EmailAddress2'].isin(email_list_20212022)]

total_comms_heliex_2021  = total_comms_2021[(total_comms_2021['EmailAddress1'].isin(email_list_20212022)) & 
               total_comms_2021['EmailAddress2'].isin(email_list_20212022)]

common_emailaddress_list = total_comms_heliex_2022[['EmailAddress1']].merge(
    total_comms_heliex_2021[['EmailAddress1']], on='EmailAddress1', how='inner')['EmailAddress1'].unique()

total_comms_heliex_2022 = total_comms_df[(total_comms_df['EmailAddress1'].isin(common_emailaddress_list)) & 
               total_comms_df['EmailAddress2'].isin(common_emailaddress_list)]

total_comms_heliex_2021  = total_comms_2021[(total_comms_2021['EmailAddress1'].isin(common_emailaddress_list)) & 
               total_comms_2021['EmailAddress2'].isin(common_emailaddress_list)]

In [33]:
len(total_comms_heliex_2022['EmailAddress2'].unique())

164

In [32]:
len(total_comms_heliex_2021['EmailAddress2'].unique())

165

In [37]:
report_dir = 'C:/Users/harry/Documents/data/Project_Node/report'

total_comms_heliex_2021.to_csv(os.path.join(report_dir, 'OverallCommunication_Helix_MartoAug2021.csv'), index=False)
total_comms_heliex_2022.to_csv(os.path.join(report_dir, 'OverallCommunication_Helix_MartoAug2022.csv'), index=False)


In [35]:
print(total_comms_heliex_2021.shape, total_comms_heliex_2022.shape)

(7231, 13) (7369, 13)
